# Plan of action

* Instead of doing manually we will use NN module for forward pass.
* We will use built in loss funtion.
* Using NN module for sigmoid function.
* For optimization we will be using optimizer.

# The nn module

The torch.nn module in PyTorch is a core library that provides a wide array of classes and
functions designed to help developers build neural networks efficiently and effectively. It
abstracts the complexity of creating and training neural networks by offering pre-built layers,
loss functions, activation functions, and other utilities, enabling you to focus on designing and
experimenting with model architectures.

Key Components of torch.nn:
1. Modules (Layers):
* nn.Module: The base class for all neural network modules. Your custom models and layers should subclass this class.
* Common Layers: Includes layers like nn.Linear (fully connected layer), nn.Conv2d (convolutional layer), nn.LSTM (recurrent layer), and many others.

2. Activation Functions:
* Functions like nn.ReLU, nn.Sigmoid, and nn.Tanh introduce non-linearities to the model, allowing it to learn complex patterns.

3. Loss Functions:
* Provides loss functions such as nn.CrossEntropyLoss, nn.MSELoss, and nn.NLLLoss to quantify the difference between the model's predictions and the actual targets.

4. Container Modules:
* nn.Sequential: A sequential container to stack layers in order.

5. Regularization and Dropout:
* Layers like nn.Dropout and nn.BatchNorm2d help prevent overfitting and improve the model's ability to generalize to new data.

In [1]:
import torch
import torch.nn as nn

# Why Use super().__init__() in PyTorch?
1. Properly Initializes the Base Class (nn.Module)

* PyTorch’s nn.Module is the base class that provides all the essential machinery for neural networks, like tracking parameters, registering submodules, and enabling features such as .to(), .cuda(), and saving/loading models.

* When you call super().__init__(), you’re making sure that all this important setup in nn.Module happens for your model. If you skip this step, your model won’t work correctly-parameters might not be registered, and PyTorch features could break.

2. Follows Object-Oriented Programming Principles

* In Python, when you create a class that inherits from another class, you should always call the parent’s __init__ method to ensure everything is set up properly. super() is the standard way to do this

In [2]:
class model(nn.Module):

  def __init__(self, num_feature):

    super().__init__()
    self.linear = nn.Linear(num_feature, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [3]:
# Create dataset
features = torch.rand(10,5)

# Create model
model = model(features.shape[1])

# Call model for forward pass.
model(features)        # model.forward(features)  --> You can also use that, but pytorch prefers other way.

tensor([[0.5729],
        [0.6366],
        [0.6105],
        [0.5662],
        [0.5553],
        [0.5540],
        [0.5654],
        [0.5748],
        [0.5885],
        [0.5966]], grad_fn=<SigmoidBackward0>)

In [4]:
model.linear.weight

Parameter containing:
tensor([[-0.1301,  0.1736,  0.0012,  0.0909,  0.0549]], requires_grad=True)

In [5]:
model.linear.bias

Parameter containing:
tensor([0.2783], requires_grad=True)

In [6]:
!pip install torchinfo

In [7]:
from torchinfo import summary

summary(model, input_size=features.shape)

Layer (type:depth-idx)                   Output Shape              Param #
model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Create a NN with a hidder layer.

* 1st layer 3 nodes. --> relu activation function.
* 2nd layer 1 nodes. --> sigmoid activation function.

In [11]:
class model1(nn.Module):

  def __init__(self, num_feature):

    super().__init__()
    self.layer1 = nn.Linear(num_feature, 3)
    self.relu = nn.ReLU()
    self.layer2 = nn.Linear(3, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.layer1(features)
    out = self.relu(out)
    out = self.layer2(out)
    out = self.sigmoid(out)

    return out


In [15]:
model = model1(features.shape[1])

model(features)

tensor([[0.4213],
        [0.4830],
        [0.4714],
        [0.4100],
        [0.4149],
        [0.4050],
        [0.4092],
        [0.4198],
        [0.4274],
        [0.4398]], grad_fn=<SigmoidBackward0>)

In [16]:
model.layer1.weight

Parameter containing:
tensor([[ 0.4308, -0.2790, -0.1598,  0.0133, -0.0993],
        [-0.4236,  0.1339,  0.2033, -0.3611, -0.3679],
        [-0.1159, -0.1987,  0.2981,  0.0021,  0.0996]], requires_grad=True)

In [17]:
model.layer1.bias

Parameter containing:
tensor([ 0.4050, -0.0512,  0.1821], requires_grad=True)

In [18]:
summary(model, input_size=features.shape)

Layer (type:depth-idx)                   Output Shape              Param #
model1                                   [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Sequential container.

Why We Need nn.Sequential
1. Simplifies Model Definition

* Instead of writing a custom class and manually defining the forward() method, you can stack layers in the order you want them to run. PyTorch will automatically connect the output of one layer to the input of the next.

2. Treats the Whole Sequence as a Single Module

* The entire sequence of layers acts as one module. This means you can easily move it to a device (like GPU), save/load it, or use it as a part of a bigger model

In [29]:
class model2(nn.Module):

  def __init__(self, num_feature):

    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_feature, 3),
        nn.ReLU(),
        nn.Linear(3,1),
        nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)
    return out

In [30]:
features = torch.rand(10,5)
print(features.shape[1])
model2 = model2(5)

model2(features)

5


tensor([[0.4456],
        [0.4888],
        [0.4039],
        [0.4899],
        [0.4565],
        [0.4632],
        [0.4253],
        [0.4690],
        [0.4308],
        [0.4709]], grad_fn=<SigmoidBackward0>)